Create a an LLM pipeline that will transform any free text query into a SQL query, the key points of this task are:

* Create a valid representation of SQL tables allowing for semantic search that will match the top results to the given free text query

* Based on the table representation the LLM has to create a real SQL query, based on free text user query, that will allow for immediate usage

* LLM has to support creation of queries with different levels of complexity not only the simplest ones

* LLM has to support creating queries to fetch data from different database schemas

* When an error in LLM created sql query is encountered it should attempt to self correct

In [1]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import sqlparse
import subprocess
import csv

In [2]:
llm = LlamaCpp(model_path="sqlcoder2-GGUF\sqlcoder2.Q8_0.gguf",
               n_batch=512,
               n_ctx=2048,
               n_gpu_layers=30,
               verbose=True)

llama_model_loader: loaded meta data with 19 key-value pairs and 485 tensors from sqlcoder2-GGUF\sqlcoder2.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = starcoder
llama_model_loader: - kv   1:                               general.name str              = StarCoder
llama_model_loader: - kv   2:                   starcoder.context_length u32              = 8192
llama_model_loader: - kv   3:                 starcoder.embedding_length u32              = 6144
llama_model_loader: - kv   4:              starcoder.feed_forward_length u32              = 24576
llama_model_loader: - kv   5:                      starcoder.block_count u32              = 40
llama_model_loader: - kv   6:             starcoder.attention.head_count u32              = 48
llama_model_loader: - kv   7:          starcoder.attention.head_count_kv u32     

In [3]:
generation_template = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Generate a SQL query to answer this question: `{user_question}`
If the question does not match any existing tables or columns, return the word 'error' without generating a SQL query.

DDL statements:
{ddl_statements}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The following SQL query best answers the question `{user_question}`:
"""

In [4]:
verification_template = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Verify if this SQL query correctly answers the question: {user_question}.
SQL query: {sql_query}
If yes, return the same query. If not return corrected query.

DDL statements:
{ddl_statements}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The following SQL query best answers the question `{user_question}`:
```sql
"""

In [5]:
def is_sql_query_valid(sql_query):
    """
    Checks if the SQL query is valid based on the content of the query.
    Returns True if valid, False if an error is detected.
    """
    if "error" in sql_query.lower():
        return False

    return True

In [6]:
generation_prompt = PromptTemplate(template=generation_template, input_variables=["user_question", "ddl_statements"])

In [7]:
verification_prompt = PromptTemplate(template=verification_template, input_variables=["user_question", "sql_query", "ddl_statements"])

In [8]:
def read_ddl_from_file(file_path):
    with open(file_path, 'r') as file:
        ddl_statements = file.read()
    return ddl_statements

In [9]:
def get_sql_query_from_llm(prompt, llm, user_question, ddl_statements):
    llm_chain = prompt | llm
    raw_llm_answer = llm_chain.invoke({"user_question": user_question, "ddl_statements": ddl_statements})
    return raw_llm_answer.strip()
    

In [10]:
def is_query_correct(sql_query):
    """
    Validate and fix a SQL query using sqlfluff. Return True if the query is correct after fixing,
    otherwise return False.
    """
    # Print the original SQL query (for debugging purposes)
    print("Original SQL Query:")
    print(sql_query)
    print("\n")

    # Write the SQL query to a temporary file
    with open("temp_query.sql", "w") as f:
        f.write(sql_query)
    
    # Run sqlfluff fix on the temporary file
    try:
        fix_result = subprocess.run(
            ["sqlfluff", "fix", "temp_query.sql", "--dialect", "ansi"],
            capture_output=True,
            text=True
        )
        
        # Print the fixed SQL query (for debugging purposes)
        with open("temp_query.sql", "r") as f:
            fixed_sql_query = f.read()

        print("Fixed SQL Query:")
        print(fixed_sql_query)
        print("\n")
        
        # Run sqlfluff lint on the fixed file
        lint_result = subprocess.run(
            ["sqlfluff", "lint", "temp_query.sql", "--dialect", "ansi"],
            capture_output=True,
            text=True
        )
        
        if lint_result.returncode == 0:
            # If lint passes, return True
            return True
        else:
            # If lint fails, check for errors and return False
            output_lines = lint_result.stderr.splitlines()
            errors = [line for line in output_lines if "error" in line.lower()]
            
            if errors:
                print("Errors found in the SQL query:")
                print("\n".join(errors))
            
            return False
    
    except Exception as e:
        print(f"Error running sqlfluff: {e}")
        return False

In [11]:
def verify_and_correct_sql(sql_query, user_question, ddl_statements):
    if not is_query_correct(sql_query):
        llm_chain = verification_prompt | llm
        raw_llm_answer = llm_chain.invoke({"user_question": user_question, "sql_query": sql_query, "ddl_statements": ddl_statements})
        return raw_llm_answer.strip()
    else:
        return sql_query

In [12]:
def text_to_sql_pipe(user_question, ddl_file_path="database.sql"):
    # Read the DDL statements from the .sql file
    ddl_statements = read_ddl_from_file(ddl_file_path)
    
    # Generate the initial SQL query
    sql_query = get_sql_query_from_llm(generation_prompt, llm, user_question, ddl_statements)
    print(f"Generated SQL Query:\n{sql_query}\n")
    
    if is_sql_query_valid(sql_query):
        # Verify and potentially correct the SQL query
        final_sql_query = verify_and_correct_sql(sql_query, user_question, ddl_statements)
        print(f"Final SQL Query After Verification and Correction:\n{final_sql_query}\n")
        
        return final_sql_query
    
    else:
        return "The query does not match any existing tables. Please check the table names or columns and try again."

In [13]:
def save_queries_to_csv(queries, filename):
    """
    Save the text queries and their corresponding SQL results to a CSV file.
    """
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(["Text Query", "Generated SQL Query"])
        
        for text_query in queries:
            # Generate SQL query from text query
            generated_sql = text_to_sql_pipe(text_query)
            
            # Write the text query and the generated SQL query to the CSV
            writer.writerow([text_query, generated_sql])
            print(f"Saved query: {text_query}\nGenerated SQL:\n{generated_sql}\n")

In [14]:
queries = [
    "Show me the names of employees and their departments.",
    "What is the highest salary?",
    "List all orders that were made in the last 30 days.",
    "What are the total sales grouped by region?",
    "List employees whose salary is greater than $50,000.",
    "Get the names of customers and their lifetime value.",
    "Show products where the sales quantity is above 100.",
    "Which departments are managed by the employee with ID 5?",
    "List all sales representatives working in the 'West' region.",
    "What is the most recent sales report date?",
    "Find the average salary of employees in each department.",
    "List customers who have placed more than 5 orders.",
    "Find the total revenue generated by each product.",
    "Show the department with the highest number of employees.",
    "Get the total number of sales representatives in each region.",
    "Find the average lifetime value of customers grouped by region.",
    "List the top 3 products with the highest revenue.",
    "Find employees who have not received a salary in the last year.",
    "Show the total number of orders per customer.",
    "List all departments with more than one manager."
]


In [15]:
csv_filename = "sql_queries_results.csv"
save_queries_to_csv(queries, csv_filename)


llama_print_timings:        load time =   38041.23 ms
llama_print_timings:      sample time =     178.35 ms /    45 runs   (    3.96 ms per token,   252.31 tokens per second)
llama_print_timings: prompt eval time =   58007.48 ms /   681 tokens (   85.18 ms per token,    11.74 tokens per second)
llama_print_timings:        eval time =  178087.58 ms /    44 runs   ( 4047.45 ms per token,     0.25 tokens per second)
llama_print_timings:       total time =  290884.22 ms /   725 tokens


Generated SQL Query:
SELECT e.first_name, e.last_name, d.department_name FROM sales.employees AS e JOIN sales.departments AS d ON e.department_id = d.department_id;

Original SQL Query:
SELECT e.first_name, e.last_name, d.department_name FROM sales.employees AS e JOIN sales.departments AS d ON e.department_id = d.department_id;


Fixed SQL Query:
SELECT
    e.first_name,
    e.last_name,
    d.department_name
FROM sales.employees AS e
INNER JOIN sales.departments AS d ON e.department_id = d.department_id;



Final SQL Query After Verification and Correction:
SELECT e.first_name, e.last_name, d.department_name FROM sales.employees AS e JOIN sales.departments AS d ON e.department_id = d.department_id;

Saved query: Show me the names of employees and their departments.
Generated SQL:
SELECT e.first_name, e.last_name, d.department_name FROM sales.employees AS e JOIN sales.departments AS d ON e.department_id = d.department_id;



Llama.generate: prefix-match hit

llama_print_timings:        load time =   38041.23 ms
llama_print_timings:      sample time =       3.23 ms /    23 runs   (    0.14 ms per token,  7118.54 tokens per second)
llama_print_timings: prompt eval time =   36167.02 ms /   632 tokens (   57.23 ms per token,    17.47 tokens per second)
llama_print_timings:        eval time =    7847.61 ms /    22 runs   (  356.71 ms per token,     2.80 tokens per second)
llama_print_timings:       total time =   46556.98 ms /   654 tokens


Generated SQL Query:
SELECT MAX(salaries.salary_amount::TEXT::INT) AS max_salary FROM salaries;

Original SQL Query:
SELECT MAX(salaries.salary_amount::TEXT::INT) AS max_salary FROM salaries;


Fixed SQL Query:
SELECT MAX(salaries.salary_amount::TEXT::INT) AS max_salary FROM salaries;



Final SQL Query After Verification and Correction:
SELECT MAX(salaries.salary_amount::TEXT::INT) AS max_salary FROM salaries;

Saved query: What is the highest salary?
Generated SQL:
SELECT MAX(salaries.salary_amount::TEXT::INT) AS max_salary FROM salaries;



Llama.generate: prefix-match hit

llama_print_timings:        load time =   38041.23 ms
llama_print_timings:      sample time =       7.05 ms /    51 runs   (    0.14 ms per token,  7236.10 tokens per second)
llama_print_timings: prompt eval time =   44680.83 ms /   648 tokens (   68.95 ms per token,    14.50 tokens per second)
llama_print_timings:        eval time =   20028.41 ms /    50 runs   (  400.57 ms per token,     2.50 tokens per second)
llama_print_timings:       total time =   64961.53 ms /   698 tokens


...

Generated SQL Query:
SELECT e."employee_id", e."first_name", e."last_name" FROM sales.employees AS e WHERE NOT EXISTS (SELECT 1 FROM sales.salaries s WHERE e."employee_id" = s."employee_id" AND s."effective_date" >= CURRENT_DATE - INTERVAL '1 year') ORDER BY e."first_name", e."last_name";

Original SQL Query:
SELECT e."employee_id", e."first_name", e."last_name" FROM sales.employees AS e WHERE NOT EXISTS (SELECT 1 FROM sales.salaries s WHERE e."employee_id" = s."employee_id" AND s."effective_date" >= CURRENT_DATE - INTERVAL '1 year') ORDER BY e."first_name", e."last_name";


Fixed SQL Query:
SELECT
    e.employee_id,
    e.first_name,
    e.last_name
FROM sales.employees AS e
WHERE
    NOT EXISTS (
        SELECT 1
        FROM sales.salaries AS s
        WHERE
            e.employee_id = s.employee_id
            AND s.effective_date >= CURRENT_DATE - INTERVAL '1 year'
    )
ORDER BY e.first_name, e.last_name;



Final SQL Query After Verification and Correction:
SELECT e."employee_id

Llama.generate: prefix-match hit

llama_print_timings:        load time =   38041.23 ms
llama_print_timings:      sample time =       5.28 ms /    38 runs   (    0.14 ms per token,  7194.24 tokens per second)
llama_print_timings: prompt eval time =   41970.33 ms /   638 tokens (   65.78 ms per token,    15.20 tokens per second)
llama_print_timings:        eval time =   13593.55 ms /    37 runs   (  367.39 ms per token,     2.72 tokens per second)
llama_print_timings:       total time =   55608.21 ms /   675 tokens


Generated SQL Query:
SELECT sales.orders.customer_id, COUNT(sales.orders.order_id) AS order_count FROM sales.orders GROUP BY sales.orders.customer_id;

Original SQL Query:
SELECT sales.orders.customer_id, COUNT(sales.orders.order_id) AS order_count FROM sales.orders GROUP BY sales.orders.customer_id;


Fixed SQL Query:
SELECT
    sales.orders.customer_id,
    COUNT(sales.orders.order_id) AS order_count
FROM sales.orders
GROUP BY sales.orders.customer_id;



Final SQL Query After Verification and Correction:
SELECT sales.orders.customer_id, COUNT(sales.orders.order_id) AS order_count FROM sales.orders GROUP BY sales.orders.customer_id;

Saved query: Show the total number of orders per customer.
Generated SQL:
SELECT sales.orders.customer_id, COUNT(sales.orders.order_id) AS order_count FROM sales.orders GROUP BY sales.orders.customer_id;



Llama.generate: prefix-match hit

llama_print_timings:        load time =   38041.23 ms
llama_print_timings:      sample time =       5.42 ms /    37 runs   (    0.15 ms per token,  6830.35 tokens per second)
llama_print_timings: prompt eval time =   54940.12 ms /   640 tokens (   85.84 ms per token,    11.65 tokens per second)
llama_print_timings:        eval time =   19778.38 ms /    36 runs   (  549.40 ms per token,     1.82 tokens per second)
llama_print_timings:       total time =   74811.94 ms /   676 tokens


Generated SQL Query:
SELECT department_name, COUNT(manager_id) AS managers FROM public.departments GROUP BY department_name HAVING COUNT(manager_id) > 1;

Original SQL Query:
SELECT department_name, COUNT(manager_id) AS managers FROM public.departments GROUP BY department_name HAVING COUNT(manager_id) > 1;


Fixed SQL Query:
SELECT
    department_name,
    COUNT(manager_id) AS managers
FROM public.departments
GROUP BY department_name
HAVING COUNT(manager_id) > 1;



Final SQL Query After Verification and Correction:
SELECT department_name, COUNT(manager_id) AS managers FROM public.departments GROUP BY department_name HAVING COUNT(manager_id) > 1;

Saved query: List all departments with more than one manager.
Generated SQL:
SELECT department_name, COUNT(manager_id) AS managers FROM public.departments GROUP BY department_name HAVING COUNT(manager_id) > 1;

